In [1]:
# Read the train and test files
import pandas as pd
train = pd.read_csv("train_E6oV3lV.csv")
test = pd.read_csv("test_tweets_anuFYb8.csv")

In [2]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
#Remove hashtags, user tags, characters other than numbers, alphabets and replace them with spaces
def remove_pattern(input_txt):
    input_txt = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",input_txt).split())
    return input_txt  

In [4]:
import numpy as np
import re
#use the above function to tidy up the tweet text for train dataset
train['tidy_tweet'] = np.vectorize(remove_pattern)(train['tweet'])


In [5]:
#Do the same for test dataset
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['tweet'])

In [6]:
#convert the entire tweet text to lower for train dataset
train["tidy_tweet"] = train["tidy_tweet"].str.lower()


In [7]:
#convert the entire tweet text to lower for test dataset
test["tidy_tweet"] = test["tidy_tweet"].str.lower()

In [8]:
#remove numbers and replace them with empty string for train dataset
train["tidy_tweet"] = train["tidy_tweet"].str.replace('\d+', '')

In [9]:
#remove numbers and replace them with empty string for test dataset
test["tidy_tweet"] = test["tidy_tweet"].str.replace('\d+', '')

In [10]:
train["tidy_tweet"] = train["tidy_tweet"].str.replace(r'\b\w{1,2}\b', '')

In [11]:
test["tidy_tweet"] = test["tidy_tweet"].str.replace(r'\b\w{1,2}\b', '')

In [12]:
train["tidy_tweet"].head(15)

0     when  father  dysfunctional and   selfish  dra...
1     thanks for lyft credit  can  use cause they do...
2                                   bihday your majesty
3                model  love  take with  all the time  
4                     factsguide society now motivation
5       huge fan fare and big talking before they le...
6                                camping tomorrow danny
7     the next school year  the year for exams can  ...
8      won love the land allin cavs champions clevel...
9                                    welcome here      
10    ireland consumer price index mom climbed from ...
11     are  selfish orlando standwithorlando pulsesh...
12                get  see  daddy today days gettingfed
13    cnn calls michigan middle school build the wal...
14     comment  australia opkillingbay seashepherd h...
Name: tidy_tweet, dtype: object

In [13]:
#remove stopwords from test dataset
from nltk.corpus import stopwords
stop = stopwords.words('english')
train["tidy_tweet"] = [' '.join(word for word in sentence.split(' ') if word not in stop) for sentence in train["tidy_tweet"]]

In [14]:
#remove stopwords from test dataset
from nltk.corpus import stopwords
stop = stopwords.words('english')
test["tidy_tweet"] = [' '.join(word for word in sentence.split(' ') if word not in stop) for sentence in test["tidy_tweet"]]

In [15]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

tokenized_tweet = train["tidy_tweet"].apply(lambda x:x.split(' '))
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    train['tidy_tweet'] = tokenized_tweet

In [16]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

tokenized_tweet = test["tidy_tweet"].apply(lambda x:x.split(' '))
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    test['tidy_tweet'] = tokenized_tweet

In [17]:
# #stem all the words in tweet test for train dataset
# tokenized_tweet = test["tidy_tweet"].apply(lambda x:x.split(' '))
# tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
# for i in range(len(tokenized_tweet)):
#     tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

# test['tidy_tweet'] = tokenized_tweet

In [18]:
#import the necessary modules
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

In [19]:
#use k (7) fold cross validation
k_fold = KFold(n_splits=11)

In [20]:
k_fold.get_n_splits(train)

11

In [21]:
tfidf = TfidfVectorizer(use_idf=True, smooth_idf=True, ngram_range=(1,2), max_df = 0.5, min_df=1,  lowercase=True)

In [22]:
nb = MultinomialNB(alpha=0.01)

In [23]:
# #Use a pipeline for tfidf, naive bayes
# pipeline = Pipeline([
#     ('Vect', CountVectorizer(stop_words='english', lowercase=True,ngram_range=(1,2))),
#     ('tfidf', TfidfTransformer(use_idf=True, smooth_idf=True)),
#     ('clf', MultinomialNB(alpha=0.01))
# ])

In [29]:
confusion = np.array([[0,0],[0,0]])
scores = []
#split the train data into train data and validation data and fit the train data into pipeline and predict the validation set usig pipeline
for train_indices, test_indices in k_fold.split(train):
    train_text = train.iloc[train_indices]['tidy_tweet'].values
    train_y = train.iloc[train_indices]['label'].values.astype(str)
    
    test_text = train.iloc[test_indices]['tidy_tweet'].values
    test_y = train.iloc[test_indices]['label'].values.astype(str)
    train_x = tfidf.fit_transform(train_text)
    nb.fit(train_x, train_y)
#     pipeline.fit(train_text, train_y)
    test_x = tfidf.transform(test_text)
    predictions = nb.predict(test_x)
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label='0')
    scores.append(score)
    

In [34]:
tfidf.fit_transform(["hello how are you", "hello i am fine", "no i am not fine","that is ok"])

<4x21 sparse matrix of type '<class 'numpy.float64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf.transform(["fine fine"], [])

In [30]:
#print f1score and confusion matrix
print("score", (sum(scores)/len(scores)))
print("confusion_matrix")
print(confusion)

score 0.9824226409544088
confusion_matrix
[[29511   209]
 [  847  1395]]


In [ ]:
#get predictions for test dataset
predictions2 = pipeline.predict(test['tidy_tweet'])

In [ ]:
predictions2

In [ ]:
test['label'] = predictions2

In [ ]:
test.head()

In [ ]:
test_predictions = test[['id', 'label']].copy()

In [ ]:
test_predictions.head()

In [ ]:
len(test_predictions)

In [ ]:
test_predictions.to_csv('test_predictions.csv', index=False, encoding='utf-8')

In [ ]:
import pickle
pipeline_pickle = open('E:/pipeline.pickle', 'wb')
pickle.dump(pipeline, pipeline_pickle)
pipeline_pickle.close()